## Import necessary packages

In [1]:
import torch
import json
import random
import os
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import gc
from utils.utils_model import get_llama_3_8B

# Set random seed for reproducibility
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Define utils functions

In [ ]:
def load_hellaswag_jsonl(file_path):
    """Load HellaSwag data from local jsonl file"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

def load_interference_files(results_dir, n_files=10):
    """Load random interference result files"""
    print(f"Looking for interference files in: {results_dir}")
    
    if not os.path.exists(results_dir):
        print(f"Error: Directory {results_dir} does not exist!")
        return []
    
    all_files = []
    for file in os.listdir(results_dir):
        if file.endswith('.json') and 'sentence' in file:
            all_files.append(os.path.join(results_dir, file))
    
    print(f"Found {len(all_files)} interference files")
    
    if len(all_files) == 0:
        print("Error: No interference files found!")
        return []
    
    if len(all_files) < n_files:
        print(f"Warning: Only found {len(all_files)} files, using all of them")
        selected_files = all_files
    else:
        selected_files = random.sample(all_files, n_files)
    
    interference_data = []
    for file_path in selected_files:
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
                interference_data.append({
                    'file_path': file_path,
                    'data': data
                })
                print(f"Successfully loaded: {os.path.basename(file_path)}")
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            continue
    
    return interference_data

def parse_interference_vectors(interference_info):
    """Parse all interference vectors from loaded files"""
    vectors = []
    
    for interference_item in interference_info:
        file_path = interference_item['file_path']
        data = interference_item['data']
        file_name = os.path.basename(file_path)
        
        # Parse count_increase vector
        if 'count_increase' in data and data['count_increase'].get('success', False):
            best_records = data['count_increase'].get('best_records', [])
            if best_records:
                record = best_records[0]  # Take the first record
                hook_pos = record.get('hook_pos', [])
                scale = record.get('scale', 1.0)
                attack_vector = record.get('attack_vector', [])
                
                if hook_pos and attack_vector:
                    # Parse hook_pos
                    if len(hook_pos) == 2 and isinstance(hook_pos[0], int):
                        layer_id = hook_pos[0]
                        layer_type = hook_pos[1]
                    elif len(hook_pos) == 1:
                        layer_id = 0
                        layer_type = hook_pos[0]
                    else:
                        continue
                    
                    # Calculate final interference vector
                    final_vector = np.array(attack_vector) * scale
                    
                    vectors.append({
                        'file_name': file_name,
                        'vector_type': 'count_increase',
                        'layer_id': layer_id,
                        'layer_type': layer_type,
                        'vector': final_vector,
                        'scale': scale,
                        'vector_norm': float(np.linalg.norm(final_vector))
                    })
        
        # Parse prob_increase vector
        if 'prob_increase' in data and data['prob_increase'].get('success', False):
            record = data['prob_increase'].get('best_record', {})
            if record:
                hook_pos = record.get('hook_pos', [])
                scale = record.get('scale', 1.0)
                attack_vector = record.get('attack_vector', [])
                
                if hook_pos and attack_vector:
                    # Parse hook_pos
                    if len(hook_pos) == 2 and isinstance(hook_pos[0], int):
                        layer_id = hook_pos[0]
                        layer_type = hook_pos[1]
                    elif len(hook_pos) == 1:
                        layer_id = 0
                        layer_type = hook_pos[0]
                    else:
                        continue
                    
                    # Calculate final interference vector
                    final_vector = np.array(attack_vector) * scale
                    
                    vectors.append({
                        'file_name': file_name,
                        'vector_type': 'prob_increase',
                        'layer_id': layer_id,
                        'layer_type': layer_type,
                        'vector': final_vector,
                        'scale': scale,
                        'vector_norm': float(np.linalg.norm(final_vector))
                    })
    
    return vectors

def register_interference_hook(model, vector_info, scale=1.0, interference_scale=0.25):
    """Register interference hook on model and return handle"""
    layer_id = vector_info['layer_id']
    layer_type = vector_info['layer_type']
    vector = vector_info['vector']
    
    attack_vector_tensor = torch.tensor(vector, dtype=model.dtype).to(model.device)
    
    def attack_hook(module, input, output):
        """Attack hook function with scaling factor"""
        if isinstance(output, tuple):
            hidden_states = output[0]
        else:
            hidden_states = output
        # Apply interference to all positions with scaling
        hidden_states[0, :] += scale * attack_vector_tensor * interference_scale
        return output
    
    # Register hook based on hook type
    if layer_type == "att":
        handle = model.model.layers[layer_id].self_attn.register_forward_hook(attack_hook)
    elif layer_type == "mlp":
        handle = model.model.layers[layer_id].mlp.register_forward_hook(attack_hook)
    elif layer_type in ["res", "res_post"]:
        handle = model.model.layers[layer_id].register_forward_hook(attack_hook)
    elif layer_type == "res_mid":
        handle = model.model.layers[layer_id].register_forward_hook(attack_hook)
    else:
        print(f"Unknown hook type: {layer_type}")
        return None
    
    print(f"  Registered hook at Layer {layer_id} {layer_type}")
    return handle

def get_logprobs_for_choices(model, tokenizer, context, choices):
    """Calculate log probabilities for each choice"""
    logprobs = []
    
    for choice in choices:
        # Build complete text
        full_text = context + " " + choice
        
        # Encode text
        inputs = tokenizer(full_text, return_tensors="pt").to(model.device)
        context_inputs = tokenizer(context, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            # Get model outputs
            outputs = model(**inputs)
            logits = outputs.logits[0]  # [seq_len, vocab_size]
            
            # Calculate log probabilities
            log_probs = torch.log_softmax(logits, dim=-1)
            
            # Only consider tokens from the choice part
            context_len = context_inputs.input_ids.shape[1]
            choice_token_ids = inputs.input_ids[0][context_len:]
            
            # Calculate cumulative log probability for the choice part
            choice_logprob = 0.0
            for i, token_id in enumerate(choice_token_ids):
                if context_len + i < log_probs.shape[0]:
                    choice_logprob += log_probs[context_len + i - 1, token_id].item()
            
            # Normalize by number of tokens
            if len(choice_token_ids) > 0:
                choice_logprob /= len(choice_token_ids)
            
            logprobs.append(choice_logprob)
    
    return logprobs

def test_hellaswag_with_hook(model, tokenizer, test_samples, hook_handle=None, description="Testing"):
    """Test HellaSwag accuracy with hook already registered (or no hook for baseline)"""
    print(f"{description}...")
    
    correct = 0
    total = 0
    
    for example in tqdm(test_samples, desc=description, leave=False):
        try:
            context = example["ctx"]
            choices = example["endings"]
            correct_answer = int(example["label"])
            
            # Get probabilities (hook is already active if registered)
            logprobs = get_logprobs_for_choices(model, tokenizer, context, choices)
            predicted_answer = np.argmax(logprobs)
            
            if predicted_answer == correct_answer:
                correct += 1
            total += 1
            
        except Exception as e:
            print(f"Error in test: {e}")
            total += 1
            continue
    
    accuracy = correct / total if total > 0 else 0
    print(f"  {description} accuracy: {accuracy:.4f} ({correct}/{total})")
    
    return accuracy, correct, total

def clean_up_memory():
    """Clean up GPU memory"""
    torch.cuda.empty_cache()
    gc.collect()

def load_specific_interference_vector(file_path, vector_type='count_increase'):
    """Load specific interference vector from a result file"""
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # Get the specified vector type
        if vector_type == 'count_increase' and 'count_increase' in data:
            if data['count_increase'].get('success', False):
                best_records = data['count_increase'].get('best_records', [])
                if best_records:
                    record = best_records[0]  # Take the first record
                else:
                    return None
            else:
                return None
        elif vector_type == 'prob_increase' and 'prob_increase' in data:
            if data['prob_increase'].get('success', False):
                record = data['prob_increase'].get('best_record', {})
                if not record:
                    return None
            else:
                return None
        else:
            return None
        
        hook_pos = record.get('hook_pos', [])
        scale = record.get('scale', 1.0)
        attack_vector = record.get('attack_vector', [])
        
        if not hook_pos or not attack_vector:
            return None
        
        # Parse hook_pos
        if len(hook_pos) == 2 and isinstance(hook_pos[0], int):
            layer_id = hook_pos[0]
            layer_type = hook_pos[1]
        elif len(hook_pos) == 1:
            layer_id = 0
            layer_type = hook_pos[0]
        else:
            return None
        
        # Calculate final interference vector
        final_vector = np.array(attack_vector) * scale
        
        # Get the sentence
        sentence = data.get('sentence', 'Unknown sentence')
        
        vector_info = {
            'file_name': os.path.basename(file_path),
            'sentence': sentence,
            'layer_id': layer_id,
            'layer_type': layer_type,
            'vector': final_vector,
            'scale': scale,
            'vector_type': vector_type,
            'vector_norm': float(np.linalg.norm(final_vector))
        }
        
        return vector_info
        
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def get_top_k_tokens(model, tokenizer, text, top_k=10):
    """Get top-k next token predictions"""
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[0, -1, :]
        
        # Calculate probabilities
        probs = torch.softmax(logits, dim=-1)
        
        # Get top-k tokens and their probabilities
        top_probs, top_indices = torch.topk(probs, top_k)
        
        # Convert to float32 first, then to numpy to avoid bfloat16 issue
        top_probs = top_probs.float().cpu().numpy()
        top_indices = top_indices.cpu().numpy()
        
        results = []
        for i, (prob, token_id) in enumerate(zip(top_probs, top_indices)):
            token_str = tokenizer.decode([token_id])
            results.append({
                'rank': i + 1,
                'token_id': int(token_id),
                'token': token_str,
                'probability': float(prob)
            })
        
        return results

def compare_top_tokens_before_after(model, tokenizer, sentence, vector_info, top_k=10):
    """Compare top-k tokens before and after applying interference vector"""
    print(f"\n{'='*80}")
    print(f"Analyzing sentence: '{sentence}'")
    print(f"File: {vector_info['file_name']}")
    print(f"Vector: {vector_info['vector_type']} at Layer {vector_info['layer_id']} {vector_info['layer_type']}")
    print(f"Vector norm: {vector_info['vector_norm']:.4f}")
    print(f"Applied scaling: 1.0 * 0.25 = 0.25")
    print("=" * 80)
    
    # Get baseline results (no interference)
    baseline_results = get_top_k_tokens(model, tokenizer, sentence, top_k=top_k)
    
    # Register hook for interference
    hook_handle = register_interference_hook(
        model, vector_info, scale=1.0, interference_scale=0.25
    )
    
    if hook_handle is None:
        print("❌ Failed to register hook!")
        return
    
    try:
        # Get results with interference
        interference_results = get_top_k_tokens(model, tokenizer, sentence, top_k=top_k)
        
        # Print comparison table
        print(f"\nTOP-{top_k} TOKEN COMPARISON:")
        print(f"{'Rank':<4} {'Before Hook':<25} {'Prob':<12} {'After Hook':<25} {'Prob':<12} {'Change':<10}")
        print("-" * 100)
        
        for i in range(top_k):
            baseline = baseline_results[i]
            interference = interference_results[i]
            
            prob_change = interference['probability'] - baseline['probability']
            change_indicator = "↑" if prob_change > 0 else "↓" if prob_change < 0 else "="
            
            print(f"{baseline['rank']:<4} {repr(baseline['token']):<25} {baseline['probability']:<12.6f} "
                  f"{repr(interference['token']):<25} {interference['probability']:<12.6f} "
                  f"{change_indicator} {prob_change:+.6f}")
        
        # Analyze changes
        print(f"\nCHANGE ANALYSIS:")
        print("-" * 40)
        
        # Check if top token changed
        baseline_top = baseline_results[0]
        interference_top = interference_results[0]
        
        if baseline_top['token_id'] != interference_top['token_id']:
            print(f"🔄 TOP TOKEN CHANGED:")
            print(f"   Before Hook: {repr(baseline_top['token'])} (prob: {baseline_top['probability']:.6f})")
            print(f"   After Hook:  {repr(interference_top['token'])} (prob: {interference_top['probability']:.6f})")
        else:
            print(f"✅ TOP TOKEN UNCHANGED: {repr(baseline_top['token'])}")
            prob_change = interference_top['probability'] - baseline_top['probability']
            print(f"   Probability change: {prob_change:+.6f}")
        
        # Find tokens that appeared/disappeared in top-k
        baseline_tokens = {r['token_id']: r for r in baseline_results}
        interference_tokens = {r['token_id']: r for r in interference_results}
        
        new_tokens = set(interference_tokens.keys()) - set(baseline_tokens.keys())
        lost_tokens = set(baseline_tokens.keys()) - set(interference_tokens.keys())
        
        if new_tokens:
            print(f"\n📈 NEW TOKENS IN TOP-{top_k}:")
            for token_id in new_tokens:
                token_info = interference_tokens[token_id]
                print(f"   Rank {token_info['rank']}: {repr(token_info['token'])} (prob: {token_info['probability']:.6f})")
        
        if lost_tokens:
            print(f"\n📉 TOKENS DROPPED FROM TOP-{top_k}:")
            for token_id in lost_tokens:
                token_info = baseline_tokens[token_id]
                print(f"   Was rank {token_info['rank']}: {repr(token_info['token'])} (prob: {token_info['probability']:.6f})")
        
        # Calculate overall distribution change
        total_prob_change = sum(abs(interference_results[i]['probability'] - baseline_results[i]['probability']) 
                               for i in range(top_k))
        
        print(f"\n📊 OVERALL DISTRIBUTION METRICS:")
        print(f"   Total absolute probability change in top-{top_k}: {total_prob_change:.6f}")
        print(f"   Average absolute change per token: {total_prob_change/top_k:.6f}")
        print(f"   New tokens in top-{top_k}: {len(new_tokens)}")
        print(f"   Lost tokens from top-{top_k}: {len(lost_tokens)}")
        
        return {
            'baseline_results': baseline_results,
            'interference_results': interference_results,
            'top_token_changed': baseline_top['token_id'] != interference_top['token_id'],
            'total_prob_change': total_prob_change,
            'new_tokens': len(new_tokens),
            'lost_tokens': len(lost_tokens)
        }
    
    finally:
        # Remove hook
        hook_handle.remove()
        torch.cuda.empty_cache()
        gc.collect()

## Do baseline test on the raw model

In [3]:
# Load model and tokenizer
print("Loading Llama 3.1-8B-Instruct model...")
llama, tokenizer = get_llama_3_8B()

# Load HellaSwag data
jsonl_path = "./dataset/hellaswag_val.jsonl"
print(f"Loading HellaSwag data from {jsonl_path}...")

if not os.path.exists(jsonl_path):
    print(f"Error: {jsonl_path} does not exist!")
    raise FileNotFoundError(f"{jsonl_path} not found")

all_data = load_hellaswag_jsonl(jsonl_path)
print(f"Loaded {len(all_data)} samples from HellaSwag")

# Randomly select test samples
num_test_samples = 500  # Adjust this number as needed
test_samples = random.sample(all_data, min(num_test_samples, len(all_data)))
print(f"Selected {len(test_samples)} random test samples")

# Test baseline accuracy (no hook)
print("\n" + "=" * 50)
print("BASELINE TESTING")
print("=" * 50)

baseline_accuracy, baseline_correct, baseline_total = test_hellaswag_with_hook(
    llama, tokenizer, test_samples, hook_handle=None, description="Baseline Test"
)

print(f"\nBaseline Results:")
print(f"  Accuracy: {baseline_accuracy:.4f} ({baseline_accuracy*100:.2f}%)")
print(f"  Correct: {baseline_correct} / {baseline_total}")

# Clean up memory after baseline test
clean_up_memory()

Loading Llama 3.1-8B-Instruct model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading HellaSwag data from ./dataset/hellaswag_val.jsonl...
Loaded 10042 samples from HellaSwag
Selected 500 random test samples

BASELINE TESTING
Baseline Test...


  Baseline Test accuracy: 0.7720 (386/500)

Baseline Results:
  Accuracy: 0.7720 (77.20%)
  Correct: 386 / 500


## Test accuracy after intervention for each result

In [4]:
# Load interference vectors
results_dir = "./results/results_location"
n_files = 100  # Number of interference files to test

print("\n" + "=" * 50)
print("LOADING INTERFERENCE VECTORS")
print("=" * 50)

# Load interference files
interference_info = load_interference_files(results_dir, n_files=n_files)

if not interference_info:
    print("No interference files loaded. Exiting.")
else:
    # Parse all interference vectors
    vectors = parse_interference_vectors(interference_info)
    print(f"Parsed {len(vectors)} interference vectors")
    
    if vectors:
        print("\n" + "=" * 50)
        print("INTERFERENCE TESTING")
        print("=" * 50)
        
        all_results = []
        
        for i, vector_info in enumerate(vectors):
            print(f"\n--- Testing Vector {i+1}/{len(vectors)} ---")
            print(f"File: {vector_info['file_name']}")
            print(f"Vector Type: {vector_info['vector_type']}")
            print(f"Layer: {vector_info['layer_id']} {vector_info['layer_type']}")
            print(f"Scale: {vector_info['scale']}")
            print(f"Vector Norm: {vector_info['vector_norm']:.4f}")
            
            # Register interference hook ONCE for this vector
            hook_handle = register_interference_hook(
                llama, vector_info, scale=1.0, interference_scale=0.25
            )
            
            if hook_handle is None:
                print(f"  Failed to register hook for {vector_info['file_name']}")
                continue
            
            try:
                # Test with interference (hook is already active)
                interference_accuracy, interference_correct, interference_total = test_hellaswag_with_hook(
                    llama, tokenizer, test_samples, hook_handle=hook_handle, 
                    description=f"Interference Test ({vector_info['vector_type']})"
                )
                
                # Calculate accuracy drop
                accuracy_drop = baseline_accuracy - interference_accuracy
                
                result = {
                    'file_name': vector_info['file_name'],
                    'vector_type': vector_info['vector_type'],
                    'layer_id': vector_info['layer_id'],
                    'layer_type': vector_info['layer_type'],
                    'scale': vector_info['scale'],
                    'vector_norm': vector_info['vector_norm'],
                    'baseline_accuracy': baseline_accuracy,
                    'baseline_correct': baseline_correct,
                    'baseline_total': baseline_total,
                    'interference_accuracy': interference_accuracy,
                    'interference_correct': interference_correct,
                    'interference_total': interference_total,
                    'accuracy_drop': accuracy_drop
                }
                
                all_results.append(result)
                
                print(f"  Results: Baseline {baseline_accuracy:.4f} -> Interference {interference_accuracy:.4f}")
                print(f"  Accuracy drop: {accuracy_drop:.4f} ({accuracy_drop*100:+.2f} percentage points)")
                
            finally:
                # Remove hook ONCE after testing all samples
                if hook_handle:
                    hook_handle.remove()
                    print(f"  Removed hook for {vector_info['file_name']}")
                
                # Clean up memory after each vector test
                clean_up_memory()
        
        # Print summary
        print("\n" + "=" * 80)
        print("SUMMARY OF ALL INTERFERENCE TESTS")
        print("=" * 80)
        
        print(f"Baseline Accuracy: {baseline_accuracy:.4f} ({baseline_correct}/{baseline_total})")
        print(f"Tested {len(all_results)} interference vectors\n")
        
        all_drops = [r['accuracy_drop'] for r in all_results]
        
        for result in all_results:
            print(f"File: {result['file_name']}")
            print(f"  Vector Type: {result['vector_type']}")
            print(f"  Location: Layer {result['layer_id']} {result['layer_type']}")
            print(f"  Scale: {result['scale']}")
            print(f"  Vector Norm: {result['vector_norm']:.4f}")
            print(f"  Baseline: {result['baseline_accuracy']:.4f}")
            print(f"  With Interference: {result['interference_accuracy']:.4f}")
            print(f"  Accuracy Drop: {result['accuracy_drop']:.4f}")
            print("-" * 40)
        
        if all_drops:
            print(f"\nOverall Statistics:")
            print(f"  Average accuracy drop: {np.mean(all_drops):.4f}")
            print(f"  Std accuracy drop: {np.std(all_drops):.4f}")
            print(f"  Max accuracy drop: {np.max(all_drops):.4f}")
            print(f"  Min accuracy drop: {np.min(all_drops):.4f}")
            print(f"  Number of harmful vectors (positive drop): {sum(1 for x in all_drops if x > 0)}")
            print(f"  Number of beneficial vectors (negative drop): {sum(1 for x in all_drops if x < 0)}")
        
        # Save results
        output_file = "./interference_hellaswag_test_results.json"
        results_data = {
            'test_config': {
                'num_test_samples': len(test_samples),
                'num_interference_vectors': len(all_results),
                'results_directory': results_dir,
                'interference_scale': 0.25,
                'random_seed': 42
            },
            'baseline_results': {
                'accuracy': baseline_accuracy,
                'correct': baseline_correct,
                'total': baseline_total
            },
            'interference_results': all_results,
            'summary_statistics': {
                'average_accuracy_drop': np.mean(all_drops) if all_drops else 0,
                'std_accuracy_drop': np.std(all_drops) if all_drops else 0,
                'max_accuracy_drop': np.max(all_drops) if all_drops else 0,
                'min_accuracy_drop': np.min(all_drops) if all_drops else 0,
                'num_harmful_vectors': sum(1 for x in all_drops if x > 0),
                'num_beneficial_vectors': sum(1 for x in all_drops if x < 0)
            }
        }
        
        with open(output_file, 'w') as f:
            json.dump(results_data, f, indent=2)
        
        print(f"\nResults saved to {output_file}")
    
    else:
        print("No valid interference vectors found.")


LOADING INTERFERENCE VECTORS
Looking for interference files in: ./results/results_location
Found 98 interference files
Successfully loaded: location_sentence_52.json
Successfully loaded: location_sentence_33.json
Successfully loaded: location_sentence_31.json
Successfully loaded: location_sentence_100.json
Successfully loaded: location_sentence_59.json
Successfully loaded: location_sentence_91.json
Successfully loaded: location_sentence_53.json
Successfully loaded: location_sentence_70.json
Successfully loaded: location_sentence_62.json
Successfully loaded: location_sentence_15.json
Successfully loaded: location_sentence_7.json
Successfully loaded: location_sentence_72.json
Successfully loaded: location_sentence_27.json
Successfully loaded: location_sentence_34.json
Successfully loaded: location_sentence_30.json
Successfully loaded: location_sentence_95.json
Successfully loaded: location_sentence_60.json
Successfully loaded: location_sentence_81.json
Successfully loaded: location_sent

  Interference Test (count_increase) accuracy: 0.7760 (388/500)
  Results: Baseline 0.7720 -> Interference 0.7760
  Accuracy drop: -0.0040 (-0.40 percentage points)
  Removed hook for location_sentence_52.json

--- Testing Vector 2/192 ---
File: location_sentence_52.json
Vector Type: prob_increase
Layer: 0 res
Scale: -1.5
Vector Norm: 1.5010
  Registered hook at Layer 0 res
Interference Test (prob_increase)...


  Interference Test (prob_increase) accuracy: 0.7740 (387/500)
  Results: Baseline 0.7720 -> Interference 0.7740
  Accuracy drop: -0.0020 (-0.20 percentage points)
  Removed hook for location_sentence_52.json

--- Testing Vector 3/192 ---
File: location_sentence_33.json
Vector Type: count_increase
Layer: 2 res
Scale: 3.5
Vector Norm: 3.4977
  Registered hook at Layer 2 res
Interference Test (count_increase)...


  Interference Test (count_increase) accuracy: 0.7620 (381/500)
  Results: Baseline 0.7720 -> Interference 0.7620
  Accuracy drop: 0.0100 (+1.00 percentage points)
  Removed hook for location_sentence_33.json

--- Testing Vector 4/192 ---
File: location_sentence_33.json
Vector Type: prob_increase
Layer: 4 res
Scale: 14
Vector Norm: 14.0109
  Registered hook at Layer 4 res
Interference Test (prob_increase)...


  Interference Test (prob_increase) accuracy: 0.6640 (332/500)
  Results: Baseline 0.7720 -> Interference 0.6640
  Accuracy drop: 0.1080 (+10.80 percentage points)
  Removed hook for location_sentence_33.json

--- Testing Vector 5/192 ---
File: location_sentence_31.json
Vector Type: count_increase
Layer: 13 res
Scale: -10
Vector Norm: 10.0379
  Registered hook at Layer 13 res
Interference Test (count_increase)...


...

  Interference Test (prob_increase) accuracy: 0.7720 (386/500)
  Results: Baseline 0.7720 -> Interference 0.7720
  Accuracy drop: 0.0000 (+0.00 percentage points)
  Removed hook for location_sentence_64.json

--- Testing Vector 189/192 ---
File: location_sentence_66.json
Vector Type: count_increase
Layer: 2 res
Scale: -3
Vector Norm: 3.0014
  Registered hook at Layer 2 res
Interference Test (count_increase)...


  Interference Test (count_increase) accuracy: 0.7660 (383/500)
  Results: Baseline 0.7720 -> Interference 0.7660
  Accuracy drop: 0.0060 (+0.60 percentage points)
  Removed hook for location_sentence_66.json

--- Testing Vector 190/192 ---
File: location_sentence_66.json
Vector Type: prob_increase
Layer: 3 res
Scale: 4.5
Vector Norm: 4.4919
  Registered hook at Layer 3 res
Interference Test (prob_increase)...


  Interference Test (prob_increase) accuracy: 0.7760 (388/500)
  Results: Baseline 0.7720 -> Interference 0.7760
  Accuracy drop: -0.0040 (-0.40 percentage points)
  Removed hook for location_sentence_66.json

--- Testing Vector 191/192 ---
File: location_sentence_24.json
Vector Type: count_increase
Layer: 5 res
Scale: -10
Vector Norm: 10.0137
  Registered hook at Layer 5 res
Interference Test (count_increase)...


  Interference Test (count_increase) accuracy: 0.7460 (373/500)
  Results: Baseline 0.7720 -> Interference 0.7460
  Accuracy drop: 0.0260 (+2.60 percentage points)
  Removed hook for location_sentence_24.json

--- Testing Vector 192/192 ---
File: location_sentence_24.json
Vector Type: prob_increase
Layer: 12 res
Scale: 14
Vector Norm: 13.9662
  Registered hook at Layer 12 res
Interference Test (prob_increase)...


  Interference Test (prob_increase) accuracy: 0.7680 (384/500)
  Results: Baseline 0.7720 -> Interference 0.7680
  Accuracy drop: 0.0040 (+0.40 percentage points)
  Removed hook for location_sentence_24.json

SUMMARY OF ALL INTERFERENCE TESTS
Baseline Accuracy: 0.7720 (386/500)
Tested 192 interference vectors

File: location_sentence_52.json
  Vector Type: count_increase
  Location: Layer 15 res
  Scale: 14
  Vector Norm: 14.0270
  Baseline: 0.7720
  With Interference: 0.7760
  Accuracy Drop: -0.0040
----------------------------------------
File: location_sentence_52.json
  Vector Type: prob_increase
  Location: Layer 0 res
  Scale: -1.5
  Vector Norm: 1.5010
  Baseline: 0.7720
  With Interference: 0.7740
  Accuracy Drop: -0.0020
----------------------------------------
File: location_sentence_33.json
  Vector Type: count_increase
  Location: Layer 2 res
  Scale: 3.5
  Vector Norm: 3.4977
  Baseline: 0.7720
  With Interference: 0.7620
  Accuracy Drop: 0.0100
---------------------------

In [ ]:
## Analyze individual file results - top token changes for each sentence
import glob

# Process all location result files
results_dir = "./results/results_location"
print(f"Looking for location result files in: {results_dir}")

# Find all location sentence JSON files
pattern = os.path.join(results_dir, "location_sentence_*.json")
all_files = glob.glob(pattern)

print(f"Found {len(all_files)} location result files")

if len(all_files) == 0:
    print("❌ No location result files found!")
else:
    # Sort files for consistent processing order
    all_files.sort()
    
    # Limit processing for demonstration (remove this line to process all files)
    # all_files = all_files[:20]  # Process only first 20 files for testing
    
    # Process each file
    processed_count = 0
    error_count = 0
    analysis_results = []
    
    print("\n" + "=" * 100)
    print("ANALYZING TOP TOKEN CHANGES FOR EACH LOCATION SENTENCE")
    print("=" * 100)
    
    for i, file_path in enumerate(all_files):
        print(f"\n{'='*20} Processing File {i+1}/{len(all_files)} {'='*20}")
        print(f"File: {os.path.basename(file_path)}")
        
        try:
            # Try to load count_increase vector first
            vector_info = load_specific_interference_vector(file_path, vector_type='count_increase')
            
            if vector_info is None:
                # Try prob_increase if count_increase failed
                vector_info = load_specific_interference_vector(file_path, vector_type='prob_increase')
            
            if vector_info is None:
                print(f"❌ Could not load valid vector from {os.path.basename(file_path)}")
                error_count += 1
                continue
            
            # Get the sentence
            sentence = vector_info['sentence']
            
            if not sentence or sentence == 'Unknown sentence':
                print(f"❌ No valid sentence found in {os.path.basename(file_path)}")
                error_count += 1
                continue
            
            # Analyze this sentence with the interference vector
            result = compare_top_tokens_before_after(llama, tokenizer, sentence, vector_info, top_k=10)
            
            if result:
                analysis_results.append({
                    'file_name': vector_info['file_name'],
                    'sentence': sentence,
                    'vector_info': vector_info,
                    'analysis_result': result
                })
            
            processed_count += 1
            
            # Clean up memory periodically
            if (i + 1) % 5 == 0:
                clean_up_memory()
                print(f"\n💾 Memory cleanup completed after {i+1} files")
            
            # Optional: Add a small delay to prevent overwhelming output
            # import time
            # time.sleep(0.5)
            
        except Exception as e:
            print(f"❌ Error processing {os.path.basename(file_path)}: {e}")
            error_count += 1
            continue
    
    # Final summary statistics
    print("\n" + "=" * 100)
    print("FINAL PROCESSING SUMMARY")
    print("=" * 100)
    print(f"Total files found: {len(all_files)}")
    print(f"Successfully processed: {processed_count}")
    print(f"Errors encountered: {error_count}")
    print(f"Processing success rate: {processed_count/(len(all_files))*100:.1f}%")
    
    if analysis_results:
        # Calculate summary statistics across all processed files
        top_token_changes = sum(1 for r in analysis_results if r['analysis_result']['top_token_changed'])
        avg_prob_change = np.mean([r['analysis_result']['total_prob_change'] for r in analysis_results])
        avg_new_tokens = np.mean([r['analysis_result']['new_tokens'] for r in analysis_results])
        avg_lost_tokens = np.mean([r['analysis_result']['lost_tokens'] for r in analysis_results])
        
        print(f"\nOVERALL ANALYSIS STATISTICS:")
        print(f"  Files with top token changes: {top_token_changes} / {len(analysis_results)} ({top_token_changes/len(analysis_results)*100:.1f}%)")
        print(f"  Average total probability change: {avg_prob_change:.6f}")
        print(f"  Average new tokens in top-10: {avg_new_tokens:.2f}")
        print(f"  Average lost tokens from top-10: {avg_lost_tokens:.2f}")
        
        # Find files with the most significant changes
        sorted_by_change = sorted(analysis_results, key=lambda x: x['analysis_result']['total_prob_change'], reverse=True)
        
        print(f"\nTOP 5 FILES WITH MOST SIGNIFICANT TOKEN CHANGES:")
        for i, item in enumerate(sorted_by_change[:5]):
            result = item['analysis_result']
            print(f"{i+1}. {item['file_name']}")
            print(f"   Total prob change: {result['total_prob_change']:.6f}")
            print(f"   Top token changed: {'Yes' if result['top_token_changed'] else 'No'}")
            print(f"   New/Lost tokens: +{result['new_tokens']}/-{result['lost_tokens']}")
    
    # Final memory cleanup
    clean_up_memory()
    
    print(f"\n✅ Analysis completed! Check the detailed output above for individual file results.")

Looking for location result files in: ./results/results_location
Found 98 location result files

ANALYZING TOP TOKEN CHANGES FOR EACH LOCATION SENTENCE

==================== Processing File 1/98 ====================
File: location_sentence_1.json

Analyzing sentence: 'After months of planning, our road trip finally reached'
File: location_sentence_1.json
Vector: count_increase at Layer 7 res
Vector norm: 10.0087
Applied scaling: 1.0 * 0.25 = 0.25
  Registered hook at Layer 7 res

TOP-10 TOKEN COMPARISON:
Rank Before Hook               Prob         After Hook                Prob         Change    
----------------------------------------------------------------------------------------------------
1    ' its'                    0.660156     ' its'                    0.679688     ↑ +0.019531
2    ' the'                    0.243164     ' the'                    0.220703     ↓ -0.022461
3    ' a'                      0.022583     ' a'                      0.023193     ↑ +0.000610
4    ' it'